In [ ]:
#Dependencies
#!pip install pybliometrics
!pip install pip elsapy
#!pip install python-dotenv

In [5]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount= True)
#!ls "/content/drive/My Drive"
import os
from configparser import ConfigParser
config = ConfigParser()
config.read('/content/drive/My Drive/confs/scopus.conf')
os.environ['SCOPUS_KEY'] = config['scopus']['APIKey']


In [ ]:
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus.utils import config
config['Authentication']['APIKey'] = os.environ['SCOPUS_KEY']
print(config['Authentication']['APIKey'])

articles = ScopusSearch('ISSN 0169-023X', download=False)
articles.get_results_size()


In [20]:
global results
global results_df
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch
import os
import json

client = ElsClient(os.environ['SCOPUS_KEY'])
articles_search = ElsSearch('ISSN(0169-023X) AND PUBYEAR > 1999', 'scopus')
articles_search.execute(client, get_all = False)
results = articles_search.results
results_df  = articles_search.results_df 
print(len(results))
results[0]


1286


{'@_fa': 'true',
 'affiliation': [{'@_fa': 'true',
   'affiliation-city': 'Houston',
   'affiliation-country': 'United States',
   'affilname': 'University of Houston'}],
 'article-number': '101730',
 'citedby-count': '0',
 'dc:creator': 'Ordonez C.',
 'dc:identifier': 'SCOPUS_ID:85079901526',
 'dc:title': 'Guest Editorial—DaWaK 2018 Special Issue—Trends in Big Data Analytics',
 'eid': '2-s2.0-85079901526',
 'link': [{'@_fa': 'true',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85079901526',
   '@ref': 'self'},
  {'@_fa': 'true',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/85079901526?field=author,affiliation',
   '@ref': 'author-affiliation'},
  {'@_fa': 'true',
   '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85079901526&origin=inward',
   '@ref': 'scopus'},
  {'@_fa': 'true',
   '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85079901526&origin=inward',
   '@ref': 'scopus-citedby'},
  {'@

In [ ]:
import pandas as pd
#l1 = [ l['@href'] for l in results[0]['link'] if l['@ref']=='scopus']

pd.set_option('max_colwidth', -1)
results_df.head()


In [ ]:
import lxml.html
import requests

def get_abstract_keywords_by_pii(pii):
  try:
    article_url = 'https://www.sciencedirect.com/science/article/abs/pii/' + pii
    query_abstract = "//article[@role='main']//div[@id='abstracts']/div/div"
    query_keywords = "//article[@role='main']//div[@class='keywords-section']/div"
    header = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:78.0) Gecko/20100101 Firefox/78.0'
    }
    response = requests.get(article_url, headers=header)
    html = lxml.html.fromstring(response.content.decode('utf-8'))
    abstract = html.xpath(query_abstract)[0].text_content()
    keywords = html.xpath(query_keywords)[0].text_content()
    return (abstract, ', '.join([t.text_content() for t in html.xpath(query_keywords)]))
  except Exception as e:
    print(str(e))
    return ('Error at fetching', 'Error at fetching')

#result = get_abstract_keywords_by_pii('S0169023X19304380')
#print(result[0])
#print(result[1])

#global target_df
target_df = results_df[['dc:identifier','dc:creator','dc:title', 'prism:doi','prism:issn','prism:volume','pii', 'prism:publicationName','subtype','subtypeDescription','prism:coverDate','prism:aggregationType']]
target_df['abstract'] = target_df.apply(lambda row: get_abstract_keywords_by_pii(row['pii'])[0], axis=1 )
target_df['keywords'] = target_df.apply(lambda row: get_abstract_keywords_by_pii(row['pii'])[1], axis=1 )
target_df.to_csv('data-knowledge-articles.csv')
target_df.head()


In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
import json

client = ElsClient(os.environ['SCOPUS_KEY'])
doc = AbsDoc(scp_id = '85079901526')
#doc = AbsDoc(uri = 'https://api.elsevier.com/content/abstract/scopus_id/85079901526')
#doc = FullDoc(doi='10.1016/j.datak.2019.101730')


if doc.read(els_client=client):
   print(doc.data)
else:
   print("Nothing")
print("Request status code: %s" % client.req_status)


{'affiliation': [{'affiliation-city': 'Houston', 'affilname': 'University of Houston', 'affiliation-country': 'United States'}, {'affiliation-city': 'Futuroscope', 'affilname': 'ISAE-ENSMA', 'affiliation-country': 'France'}], 'coredata': {'srctype': 'j', 'eid': '2-s2.0-85079901526', 'prism:coverDate': '2020-03-01', 'prism:aggregationType': 'Journal', 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/85079901526', 'subtypeDescription': 'Editorial', 'dc:creator': {'author': [{'ce:given-name': 'Carlos', 'preferred-name': {'ce:given-name': 'Carlos', 'ce:initials': 'C.', 'ce:surname': 'Ordonez', 'ce:indexed-name': 'Ordonez C.'}, '@seq': '1', 'ce:initials': 'C.', '@_fa': 'true', 'affiliation': {'@id': '60005837', '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60005837'}, 'ce:surname': 'Ordonez', '@auid': '57203051107', 'author-url': 'https://api.elsevier.com/content/author/author_id/57203051107', 'ce:indexed-name': 'Ordonez C.'}]}, 'link': [{'@_fa': 'tr